In [2]:
!pip install streamlit

  Using cached streamlit-1.31.0-py2.py3-none-any.whl (8.4 MB)
  Using cached rich-13.7.0-py3-none-any.whl (240 kB)
  Using cached altair-5.2.0-py3-none-any.whl (996 kB)
  Using cached blinker-1.7.0-py3-none-any.whl (13 kB)
  Using cached pyarrow-15.0.0-cp39-cp39-win_amd64.whl (24.9 MB)
  Using cached tzlocal-5.2-py3-none-any.whl (17 kB)
  Using cached validators-0.22.0-py3-none-any.whl (26 kB)
  Using cached GitPython-3.1.41-py3-none-any.whl (196 kB)
  Using cached pydeck-0.8.1b0-py2.py3-none-any.whl (4.8 MB)
  Using cached tenacity-8.2.3-py3-none-any.whl (24 kB)
  Using cached protobuf-4.25.2-cp39-cp39-win_amd64.whl (413 kB)
  Using cached gitdb-4.0.11-py3-none-any.whl (62 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl (87 kB)
  Attempting uninstall: tenacity
    Found existing installation: tenacity 8.0.1
    Uninstalling tenacity-8.0.1:
      Successfully uninstalled tenacity-8.0.1
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.1
    Uninst

In [3]:
import streamlit

In [7]:
import streamlit as st
import pandas as pd
import requests

# URL de l'API Flask
API_URL = "http://127.0.0.1:8080/predict"

# Interface utilisateur Streamlit
st.title("Application de Prédiction de la consommation d'énergie des bâtiments non résidentiels")


user_inputs = {}

# Charger la base de données depuis le fichier CSV
data = pd.read_csv("C:/Users/DELL/Downloads/projet_ml/data/2016_Building_Energy_final.csv", sep=",")

# Récupérer la liste des noms de colonnes (variables)
variable_names = data.columns.tolist()
variable_names.remove('SiteEnergyUse(kBtu)')

YearBuilt = None

# Générer automatiquement les champs de saisie
for variable_name in variable_names:
    # Si la variable est 'SiteEnergyUse', utiliser un selectbox
    if len(data[variable_name].unique()) <= 13 and variable_name != 'NumberOfPropertyUseTypes' and variable_name != 'YearBuilt' and variable_name != 'NombreAnnees' and variable_name != 'NumberofBuildings':
        selected_value = st.selectbox(variable_name, data[variable_name].unique())
    # Si la variable a plus de 12 valeurs uniques et n'est pas 'SiteEnergyUse(kBtu)', et est numérique
    elif len(data[variable_name].unique()) > 13 and variable_name != 'NumberOfPropertyUseTypes' and variable_name != 'YearBuilt' and variable_name != 'NombreAnnees' and variable_name != 'NumberofBuildings' and pd.api.types.is_numeric_dtype(data[variable_name]):
        selected_value = st.number_input(variable_name, min_value=0)
    elif variable_name == 'NumberOfPropertyUseTypes':
        selected_value = st.number_input(variable_name, min_value=1, max_value=10)
    elif variable_name == 'NumberofBuildings':
        selected_value = st.number_input(variable_name, min_value=1, max_value=50)
    # Si la variable est 'YearBuilt', utiliser un champ de saisie numérique avec des valeurs limites
    elif variable_name == 'YearBuilt':
        selected_value = st.number_input(variable_name, min_value=1800, max_value=2016)
        YearBuilt = selected_value
    # Si la variable est 'NombreAnnees', calculer automatiquement en fonction de 'YearBuilt'
    elif variable_name == 'NombreAnnees':
        selected_value = 2016 - YearBuilt if YearBuilt else None
    # Sinon, utiliser un champ de texte pour les autres variables
    else:
        selected_value = st.text_input(variable_name)

    
    # Stocker la valeur sélectionnée dans le dictionnaire
    user_inputs[variable_name] = selected_value

if st.button("Effectuer la prédiction"):
    try:
        # Collecter les valeurs saisies par l'utilisateur à partir du dictionnaire
        inputs = {
        "BuildingType" : str(user_inputs["BuildingType"]),
        "CouncilDistrictCode" : str(user_inputs["CouncilDistrictCode"]),
        "Neighborhood" : str(user_inputs["Neighborhood"]),
        "YearBuilt" : int(user_inputs["YearBuilt"]),
        "NumberofBuildings" : int(user_inputs["NumberofBuildings"]),
        "NumberofFloors" : int(user_inputs["NumberofFloors"]),
        "PropertyGFAParking" : int(user_inputs["PropertyGFAParking"]),
        "PropertyGFABuilding(s)" : int(user_inputs["PropertyGFABuilding(s)"]),
        "LargestPropertyUseTypeGFA" : int(user_inputs["LargestPropertyUseTypeGFA"]),
        "SecondLargestPropertyUseTypeGFA" : int(user_inputs["SecondLargestPropertyUseTypeGFA"]),
        "ThirdLargestPropertyUseTypeGFA" : int(user_inputs["ThirdLargestPropertyUseTypeGFA"]),
        "ENERGYSTARScore" : int(user_inputs["ENERGYSTARScore"]),
        "TotalGHGEmissions" : int(user_inputs["TotalGHGEmissions"]),
        "PrimaryProperty" : str(user_inputs["PrimaryProperty"]),
        "SecondLargest" : str(user_inputs["SecondLargest"]),
        "ThirdLargest" : str(user_inputs["ThirdLargest"]),
        "NumberOfPropertyUseTypes" : int(user_inputs["NumberOfPropertyUseTypes"]),
        "NombreAnnees" : int(user_inputs["NombreAnnees"])
    }

        # Faire une requête à l'API Flask pour obtenir la prédiction
        response = requests.post(API_URL, json=inputs)

        if response.status_code == 200:
            prediction = response.json()['prediction']
            st.success(f"La prédiction de la consommation d'énergie sur site en kBtu pour ce batiment est : {prediction}")
        else:
            st.error("Erreur lors de la prédiction.")

    except Exception as e:
        st.error(f"Erreur: {e}")

In [20]:
%run app.py